In [1]:
import os
from sympy import *
import numpy as np
import pandas as pd
init_printing()

/tmp/ipykernel_16846/1119944706.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# initialize variables
num_rlc = 0 # number of passive elements
num_ind = 0 # number of inductors
num_v = 0    # number of independent voltage sources
num_i = 0    # number of independent current sources
i_unk = 0  # number of current unknowns
num_opamps = 0   # number of op amps
num_vcvs = 0     # number of controlled sources of various types
num_vccs = 0
num_cccs = 0
num_ccvs = 0
num_cpld_ind = 0 # number of coupled inductors

In [3]:
fn = 'test_14a'
#fn = 'Chua1-b' # net list
#fn = 'state_variable_filter-1' # net list
#fn = 'example-rc' # net list
#fn = 'Two amplifier RIAA Phone Preamp w edits' # net list
#fn = 'SMNA_example_net' # net list
fd1 = open('ldo_macromodel.spice','r')
content = fd1.readlines()
content = [x.strip() for x in content]  #remove leading and trailing white space
# remove empty lines
while '' in content:
    content.pop(content.index(''))

# remove comment lines, these start with a asterisk *
content = [n for n in content if not n.startswith('*')]
# remove other comment lines, these start with a semicolon ;
content = [n for n in content if not n.startswith(';')]
# remove spice directives, these start with a period, .
content = [n for n in content if not n.startswith('.')]
# converts 1st letter to upper case
#content = [x.upper() for x in content] <- this converts all to upper case
content = [x.capitalize() for x in content]
# removes extra spaces between entries
content = [' '.join(x.split()) for x in content]

In [4]:
line_cnt = len(content) # number of lines in the netlist
branch_cnt = 0  # number of branches in the netlist
# check number of entries on each line, count each element type
for i in range(line_cnt):
    x = content[i][0]
    tk_cnt = len(content[i].split()) # split the line into a list of words

    if (x == 'R') or (x == 'L') or (x == 'C'):
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_rlc += 1
        branch_cnt += 1
        if x == 'L':
            num_ind += 1
    elif x == 'V':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_v += 1
        branch_cnt += 1
    elif x == 'I':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_i += 1
        branch_cnt += 1
    elif x == 'O':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_opamps += 1
    elif x == 'E':
        if (tk_cnt != 6):
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 6".format(tk_cnt))
        num_vcvs += 1
        branch_cnt += 1
    elif x == 'G':
        if (tk_cnt != 6):
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 6".format(tk_cnt))
        num_vccs += 1
        branch_cnt += 1
    elif x == 'F':
        if (tk_cnt != 5):
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 5".format(tk_cnt))
        num_cccs += 1
        branch_cnt += 1
    elif x == 'H':
        if (tk_cnt != 5):
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 5".format(tk_cnt))
        num_ccvs += 1
        branch_cnt += 1
    elif x == 'K':
        if (tk_cnt != 4):
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_cpld_ind += 1
    else:
        print("unknown element type in branch {:d}, {:s}".format(i,content[i]))

In [5]:
# build the pandas data frame
df = pd.DataFrame(columns=['element','p node','n node','cp node','cn node',
    'Vout','value','Vname','Lname1','Lname2'])

# this data frame is for branches with unknown currents
df2 = pd.DataFrame(columns=['element','p node','n node'])

In [6]:
# loads voltage or current sources into branch structure
def indep_source(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'value'] = float(tk[3])

# loads passive elements into branch structure
def rlc_element(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'value'] = np.format_float_scientific(np.float32(tk[3]))

# loads multi-terminal elements into branch structure
# O - Op Amps
def opamp_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'Vout'] = int(tk[3])

# G - VCCS
def vccs_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'cp node'] = int(tk[3])
    df.loc[line_nu,'cn node'] = int(tk[4])
    df.loc[line_nu,'value'] = np.format_float_scientific(float(tk[5]))

# E - VCVS
# in sympy E is the number 2.718, replacing E with Ea otherwise, sympify() errors out
def vcvs_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0].replace('E', 'Ea')
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'cp node'] = int(tk[3])
    df.loc[line_nu,'cn node'] = int(tk[4])
    df.loc[line_nu,'value'] = float(tk[5])

# F - CCCS
def cccs_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'Vname'] = tk[3].capitalize()
    df.loc[line_nu,'value'] = float(tk[4])

# H - CCVS
def ccvs_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'Vname'] = tk[3].capitalize()
    df.loc[line_nu,'value'] = float(tk[4])

# K - Coupled inductors
def cpld_ind_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'Lname1'] = tk[1].capitalize()
    df.loc[line_nu,'Lname2'] = tk[2].capitalize()
    df.loc[line_nu,'value'] = float(tk[3])

# function to scan df and get largest node number
def count_nodes():
    # need to check that nodes are consecutive
    # fill array with node numbers
    p = np.zeros(line_cnt+1)
    for i in range(line_cnt):
        # need to skip coupled inductor 'K' statements
        if df.loc[i,'element'][0] != 'K': #get 1st letter of element name
            p[df['p node'][i]] = df['p node'][i]
            p[df['n node'][i]] = df['n node'][i]

    # find the largest node number
    if df['n node'].max() > df['p node'].max():
        largest = df['n node'].max()
    else:
        largest =  df['p node'].max()

    largest = int(largest)
    # check for unfilled elements, skip node 0
    for i in range(1,largest):
        if p[i] == 0:
            print('nodes not in continuous order, node {:.0f} is missing'.format(p[i-1]+1))

    return largest

In [7]:
# load branch info into data frame
for i in range(line_cnt):
    x = content[i][0]

    if (x == 'R') or (x == 'L') or (x == 'C'):
        rlc_element(i)
    elif (x == 'V') or (x == 'I'):
        indep_source(i)
    elif x == 'O':
        opamp_sub_network(i)
    elif x == 'E':
        vcvs_sub_network(i)
    elif x == 'G':
        vccs_sub_network(i)
    elif x == 'F':
        cccs_sub_network(i)
    elif x == 'H':
        ccvs_sub_network(i)
    elif x == 'K':
        cpld_ind_sub_network(i)
    else:
        print("unknown element type in branch {:d}, {:s}".format(i,content[i]))

In [8]:
# Check for position of voltages sources in the dataframe.
source_index = [] # keep track of voltage source row number
other_index = [] # make a list of all other types
for i in range(len(df)):
    # process all the elements creating unknown currents
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if (x == 'V'):
        source_index.append(i)
    else:
        other_index.append(i)

df = df.reindex(source_index+other_index,copy=True) # re-order the data frame
df.reset_index(drop=True, inplace=True) # renumber the index

In [9]:
# count number of nodes
num_nodes = count_nodes()

# Build df2: consists of branches with current unknowns, used for C & D matrices
# walk through data frame and find these parameters
count = 0
for i in range(len(df)):
    # process all the elements creating unknown currents
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if (x == 'L') or (x == 'V') or (x == 'O') or (x == 'E') or (x == 'H') or (x == 'F'):
        df2.loc[count,'element'] = df.loc[i,'element']
        df2.loc[count,'p node'] = df.loc[i,'p node']
        df2.loc[count,'n node'] = df.loc[i,'n node']
        count += 1

In [10]:
# print a report
print('Net list report')
print('number of lines in netlist: {:d}'.format(line_cnt))
print('number of branches: {:d}'.format(branch_cnt))
print('number of nodes: {:d}'.format(num_nodes))
# count the number of element types that affect the size of the B, C, D, E and J arrays
# these are current unknows
i_unk = num_v+num_opamps+num_vcvs+num_ccvs+num_cccs+num_ind
print('number of unknown currents: {:d}'.format(i_unk))
print('number of RLC (passive components): {:d}'.format(num_rlc))
print('number of inductors: {:d}'.format(num_ind))
print('number of independent voltage sources: {:d}'.format(num_v))
print('number of independent current sources: {:d}'.format(num_i))
print('number of op amps: {:d}'.format(num_opamps))
print('number of E - VCVS: {:d}'.format(num_vcvs))
print('number of G - VCCS: {:d}'.format(num_vccs))
print('number of F - CCCS: {:d}'.format(num_cccs))
print('number of H - CCVS: {:d}'.format(num_ccvs))
print('number of K - Coupled inductors: {:d}'.format(num_cpld_ind))

Net list report
number of lines in netlist: 31
number of branches: 31
number of nodes: 7
number of unknown currents: 1
number of RLC (passive components): 23
number of inductors: 0
number of independent voltage sources: 1
number of independent current sources: 0
number of op amps: 0
number of E - VCVS: 0
number of G - VCCS: 7
number of F - CCCS: 0
number of H - CCVS: 0
number of K - Coupled inductors: 0


In [11]:
df

,element,p node,n node,cp node,cn node,Vout,value,Vname,Lname1,Lname2
0,Vdd,1,0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,Gm_xm1,3,4,0,3,NaN,1.4165539585421123e-05,NaN,NaN,NaN
2,Rds_xm1,4,3,NaN,NaN,NaN,7.9720115e+06,NaN,NaN,NaN
3,Cgs_xm1,0,4,NaN,NaN,NaN,-1.7089028e-15,NaN,NaN,NaN
4,Cgd_xm1,0,3,NaN,NaN,NaN,-2.8881355e-19,NaN,NaN,NaN
5,Gm_xm2,2,4,5,2,NaN,1.5041396621557104e-05,NaN,NaN,NaN
6,Rds_xm2,4,2,NaN,NaN,NaN,6.889675e+06,NaN,NaN,NaN
7,Cgs_xm2,5,4,NaN,NaN,NaN,-1.7287e-15,NaN,NaN,NaN
8,Cgd_xm2,5,2,NaN,NaN,NaN,-9.165741e-19,NaN,NaN,NaN
9,Gm_xm3,3,1,3,3,NaN,7.0481940867603015e-06,NaN,NaN,NaN


In [12]:
# initialize some symbolic matrix with zeros
# A is formed by [[G, C] [B, D]]
# Z = [I,E]
# X = [V, J]
V = zeros(num_nodes,1)
I = zeros(num_nodes,1)
G = zeros(num_nodes,num_nodes)  # also called Yr, the reduced nodal matrix
s = Symbol('s')  # the Laplace variable

# count the number of element types that affect the size of the B, C, D, E and J arrays
# these are element types that have unknown currents
i_unk = num_v+num_opamps+num_vcvs+num_ccvs+num_ind+num_cccs
# if i_unk == 0, just generate empty arrays
B = zeros(num_nodes,i_unk)
C = zeros(i_unk,num_nodes)
D = zeros(i_unk,i_unk)
Ev = zeros(i_unk,1)
J = zeros(i_unk,1)

In [13]:
# G matrix
for i in range(len(df)):  # process each row in the data frame
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    cn1 = df.loc[i,'cp node']
    cn2 = df.loc[i,'cn node']
    # process all the passive elements, save conductance to temp value
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'R':
        g = 1/sympify(df.loc[i,'element'])
    if x == 'C':
        g = s*sympify(df.loc[i,'element'])
    if x == 'G':   #vccs type element
        g = sympify(df.loc[i,'element'].lower())  # use a symbol for gain value

    if (x == 'R') or (x == 'C'):
        # If neither side of the element is connected to ground
        # then subtract it from the appropriate location in the matrix.
        if (n1 != 0) and (n2 != 0):
            G[n1-1,n2-1] += -g
            G[n2-1,n1-1] += -g

        # If node 1 is connected to ground, add element to diagonal of matrix
        if n1 != 0:
            G[n1-1,n1-1] += g

        # same for for node 2
        if n2 != 0:
            G[n2-1,n2-1] += g

    if x == 'G':    #vccs type element
        # check to see if any terminal is grounded
        # then stamp the matrix
        if n1 != 0 and cn1 != 0:
            G[n1-1,cn1-1] += g

        if n2 != 0 and cn2 != 0:
            G[n2-1,cn2-1] += g

        if n1 != 0 and cn2 != 0:
            G[n1-1,cn2-1] -= g

        if n2 != 0 and cn1 != 0:
            G[n2-1,cn1-1] -= g

G  # display the G matrix

⎡                                  1        1        1                        
⎢Cgsₚₜ⋅s + Cgsₓₘ₃⋅s + Cgsₓₘ₄⋅s + ────── + ────── + ─────                      
⎢                                Rdsₓₘ₄   Rdsₓₘ₃   Rdsₚₜ                      
⎢                                                                             
⎢                                1                                            
⎢                   -Cgsₚₜ⋅s - ──────                     Cgdₚₜ⋅s + Cgdₓₘ₂⋅s +
⎢                              Rdsₓₘ₄                                         
⎢                                                                             
⎢                                      1                                      
⎢             -Cgsₓₘ₃⋅s - Cgsₓₘ₄⋅s - ──────                                   
⎢                                    Rdsₓₘ₃                                   
⎢                                                                             
⎢                                                   

In [14]:
# generate the B Matrix
sn = 0   # count source number as code walks through the data frame
for i in range(len(df)):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    n_vout = df.loc[i,'Vout'] # node connected to op amp output

    # process elements with input to B matrix
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        if i_unk > 1:  #is B greater than 1 by n?, V
            if n1 != 0:
                B[n1-1,sn] = 1
            if n2 != 0:
                B[n2-1,sn] = -1
        else:
            if n1 != 0:
                B[n1-1] = 1
            if n2 != 0:
                B[n2-1] = -1
        sn += 1   #increment source count
    if x == 'O':  # op amp type, output connection of the opamp goes in the B matrix
        B[n_vout-1,sn] = 1
        sn += 1   # increment source count
    if (x == 'H') or (x == 'F'):  # H: ccvs, F: cccs,
        if i_unk > 1:  #is B greater than 1 by n?, H, F
            # check to see if any terminal is grounded
            # then stamp the matrix
            if n1 != 0:
                B[n1-1,sn] = 1
            if n2 != 0:
                B[n2-1,sn] = -1
        else:
            if n1 != 0:
                B[n1-1] = 1
            if n2 != 0:
                B[n2-1] = -1
        sn += 1   #increment source count
    if x == 'E':   # vcvs type, only ik column is altered at n1 and n2
        if i_unk > 1:  #is B greater than 1 by n?, E
            if n1 != 0:
                B[n1-1,sn] = 1
            if n2 != 0:
                B[n2-1,sn] = -1
        else:
            if n1 != 0:
                B[n1-1] = 1
            if n2 != 0:
                B[n2-1] = -1
        sn += 1   #increment source count
    if x == 'L':
        if i_unk > 1:  #is B greater than 1 by n?, L
            if n1 != 0:
                B[n1-1,sn] = 1
            if n2 != 0:
                B[n2-1,sn] = -1
        else:
            if n1 != 0:
                B[n1-1] = 1
            if n2 != 0:
                B[n2-1] = -1
        sn += 1   #increment source count

# check source count
if sn != i_unk:
    print('source number, sn={:d} not equal to i_unk={:d} in matrix B'.format(sn,i_unk))

B   # display the B matrix

⎡1⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [15]:
# find the the column position in the C and D matrix for controlled sources
# needs to return the node numbers and branch number of controlling branch
def find_vname(name):
    # need to walk through data frame and find these parameters
    for i in range(len(df2)):
        # process all the elements creating unknown currents
        if name == df2.loc[i,'element']:
            n1 = df2.loc[i,'p node']
            n2 = df2.loc[i,'n node']
            return n1, n2, i  # n1, n2 & col_num are from the branch of the controlling element

    print('failed to find matching branch element in find_vname')

In [16]:
# generate the C Matrix
sn = 0   # count source number as code walks through the data frame
for i in range(len(df)):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    cn1 = df.loc[i,'cp node'] # nodes for controlled sources
    cn2 = df.loc[i,'cn node']
    n_vout = df.loc[i,'Vout'] # node connected to op amp output

    # process elements with input to B matrix
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        if i_unk > 1:  #is B greater than 1 by n?, V
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1
        sn += 1   #increment source count

    if x == 'O':  # op amp type, input connections of the opamp go into the C matrix
        # C[sn,n_vout-1] = 1
        if i_unk > 1:  #is B greater than 1 by n?, O
            # check to see if any terminal is grounded
            # then stamp the matrix
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1
        sn += 1   # increment source count

    if x == 'F':  # need to count F (cccs) types
        sn += 1   #increment source count
    if x == 'H':  # H: ccvs
        if i_unk > 1:  #is B greater than 1 by n?, H
            # check to see if any terminal is grounded
            # then stamp the matrix
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1
        sn += 1   #increment source count
    if x == 'E':   # vcvs type, ik column is altered at n1 and n2, cn1 & cn2 get value
        if i_unk > 1:  #is B greater than 1 by n?, E
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
            # add entry for cp and cn of the controlling voltage
            if cn1 != 0:
                C[sn,cn1-1] = -sympify(df.loc[i,'element'].lower())
            if cn2 != 0:
                C[sn,cn2-1] = sympify(df.loc[i,'element'].lower())
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1
            vn1, vn2, df2_index = find_vname(df.loc[i,'Vname'])
            if vn1 != 0:
                C[vn1-1] = -sympify(df.loc[i,'element'].lower())
            if vn2 != 0:
                C[vn2-1] = sympify(df.loc[i,'element'].lower())
        sn += 1   #increment source count

    if x == 'L':
        if i_unk > 1:  #is B greater than 1 by n?, L
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1
        sn += 1   #increment source count

# check source count
if sn != i_unk:
    print('source number, sn={:d} not equal to i_unk={:d} in matrix C'.format(sn,i_unk))

C   # display the C matrix

[1  0  0  0  0  0  0]

In [17]:
# generate the D Matrix
sn = 0   # count source number as code walks through the data frame
for i in range(len(df)):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    #cn1 = df.loc[i,'cp node'] # nodes for controlled sources
    #cn2 = df.loc[i,'cn node']
    #n_vout = df.loc[i,'Vout'] # node connected to op amp output

    # process elements with input to D matrix
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if (x == 'V') or (x == 'O') or (x == 'E'):  # need to count V, E & O types
        sn += 1   #increment source count

    if x == 'L':
        if i_unk > 1:  #is D greater than 1 by 1?
            D[sn,sn] += -s*sympify(df.loc[i,'element'])
        else:
            D[sn] += -s*sympify(df.loc[i,'element'])
        sn += 1   #increment source count

    if x == 'H':  # H: ccvs
        # if there is a H type, D is m by m
        # need to find the vn for Vname
        # then stamp the matrix
        vn1, vn2, df2_index = find_vname(df.loc[i,'Vname'])
        D[sn,df2_index] += -sympify(df.loc[i,'element'].lower())
        sn += 1   #increment source count

    if x == 'F':  # F: cccs
        # if there is a F type, D is m by m
        # need to find the vn for Vname
        # then stamp the matrix
        vn1, vn2, df2_index = find_vname(df.loc[i,'Vname'])
        D[sn,df2_index] += -sympify(df.loc[i,'element'].lower())
        D[sn,sn] = 1
        sn += 1   #increment source count

    if x == 'K':  # K: coupled inductors, KXX LYY LZZ value
        # if there is a K type, D is m by m
        vn1, vn2, ind1_index = find_vname(df.loc[i,'Lname1'])  # get i_unk position for Lx
        vn1, vn2, ind2_index = find_vname(df.loc[i,'Lname2'])  # get i_unk position for Ly
        # enter sM on diagonals = value*sqrt(LXX*LZZ)

        D[ind1_index,ind2_index] += -s*sympify('M{:s}'.format(df.loc[i,'element'].lower()[1:]))  # s*Mxx
        D[ind2_index,ind1_index] += -s*sympify('M{:s}'.format(df.loc[i,'element'].lower()[1:]))  # -s*Mxx

# display the The D matrix
D

[0]

In [18]:
# generate the V matrix
for i in range(num_nodes):
    V[i] = sympify('v{:d}'.format(i+1))

V  # display the V matrix

⎡v₁⎤
⎢  ⎥
⎢v₂⎥
⎢  ⎥
⎢v₃⎥
⎢  ⎥
⎢v₄⎥
⎢  ⎥
⎢v₅⎥
⎢  ⎥
⎢v₆⎥
⎢  ⎥
⎣v₇⎦

In [19]:
# The J matrix is an m by 1 matrix, with one entry for each i_unk from a source
#sn = 0   # count i_unk source number
#oan = 0   #count op amp number
for i in range(len(df2)):
    # process all the unknown currents
    J[i] = sympify('I_{:s}'.format(df2.loc[i,'element']))

J  # diplay the J matrix

[I_Vdd]

In [20]:
# generate the I matrix, current sources have n2 = arrow end of the element
for i in range(len(df)):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the passive elements, save conductance to temp value
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'I':
        g = sympify(df.loc[i,'element'])
        # sum the current into each node
        if n1 != 0:
            I[n1-1] -= g
        if n2 != 0:
            I[n2-1] += g

I  # display the I matrix

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [21]:
# generate the E matrix
sn = 0   # count source number
for i in range(len(df)):
    # process all the passive elements
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        Ev[sn] = sympify(df.loc[i,'element'])
        sn += 1

Ev   # display the E matrix

[Vdd]

In [22]:
Z = I[:] + Ev[:]  # the + operator in python concatenates the lists
Z  # display the Z matrix

In [23]:
X = V[:] + J[:]  # the + operator in python concatenates the lists
X  # display the X matrix

In [24]:
n = num_nodes
m = i_unk
A = zeros(m+n,m+n)
for i in range(n):
    for j in range(n):
        A[i,j] = G[i,j]

if i_unk > 1:
    for i in range(n):
        for j in range(m):
            A[i,n+j] = B[i,j]
            A[n+j,i] = C[j,i]

    for i in range(m):
        for j in range(m):
            A[n+i,n+j] = D[i,j]

if i_unk == 1:
    for i in range(n):
        A[i,n] = B[i]
        A[n,i] = C[i]
    A[n,n] = D[0] # added 1/7/2024 while debugging source free circuit with one inductor

A  # display the A matrix

⎡                                  1        1        1                        
⎢Cgsₚₜ⋅s + Cgsₓₘ₃⋅s + Cgsₓₘ₄⋅s + ────── + ────── + ─────                      
⎢                                Rdsₓₘ₄   Rdsₓₘ₃   Rdsₚₜ                      
⎢                                                                             
⎢                                1                                            
⎢                   -Cgsₚₜ⋅s - ──────                     Cgdₚₜ⋅s + Cgdₓₘ₂⋅s +
⎢                              Rdsₓₘ₄                                         
⎢                                                                             
⎢                                      1                                      
⎢             -Cgsₓₘ₃⋅s - Cgsₓₘ₄⋅s - ──────                                   
⎢                                    Rdsₓₘ₃                                   
⎢                                                                             
⎢                                                   

In [25]:
equ = Eq(A*Matrix(X),Matrix(Z))
equ

⎡            ⎛                                  1        1        1  ⎞      ⎛ 
⎢ I_Vdd + v₁⋅⎜Cgsₚₜ⋅s + Cgsₓₘ₃⋅s + Cgsₓₘ₄⋅s + ────── + ────── + ─────⎟ + v₂⋅⎜-
⎢            ⎝                                Rdsₓₘ₄   Rdsₓₘ₃   Rdsₚₜ⎠      ⎝ 
⎢                                                                             
⎢                 ⎛             1   ⎞      ⎛                                  
⎢-Cgdₚₜ⋅s⋅v₇ + v₁⋅⎜-Cgsₚₜ⋅s - ──────⎟ + v₂⋅⎜Cgdₚₜ⋅s + Cgdₓₘ₂⋅s + Cgdₓₘ₄⋅s + Cg
⎢                 ⎝           Rdsₓₘ₄⎠      ⎝                                  
⎢                                                                             
⎢                                       ⎛                         1   ⎞      ⎛
⎢                     -Cgdₓₘ₄⋅s⋅v₂ + v₁⋅⎜-Cgsₓₘ₃⋅s - Cgsₓₘ₄⋅s - ──────⎟ + v₃⋅⎜
⎢                                       ⎝                       Rdsₓₘ₃⎠      ⎝
⎢                                                                             
⎢         ⎛          1   ⎞      ⎛          1   ⎞    

In [26]:
for i in range(len(X)):
    print(latex(Eq((A*Xp)[i:i+1][0],Zp[i])),'\\\\')

NameError: name 'Xp' is not defined

In [32]:
symbol_list = str(equ.free_symbols).replace('{','').replace('}','')
print(symbol_list,'= symbols(\'',symbol_list.replace(',',' '),'\')')

Cgs_xm3, v7, Cgs_pt, v4, R2, gm_xm1, I_Vdd, gm_pt, Cgs_xm4, Cgd_xm4, Cgd_xm7, Cgs_xm8, Vdd, Rds_xm7, s, Rds_xm1, Cgs_xm2, Cgs_xm1, gm_xm7, v6, Cgd_xm1, R1, Rds_xm3, v3, gm_xm2, v2, v1, Cgs_xm7, Rds_xm4, Cgd_pt, v5, Rds_xm8, Rds_xm2, Cgd_xm2, gm_xm4, Rds_pt = symbols(' Cgs_xm3  v7  Cgs_pt  v4  R2  gm_xm1  I_Vdd  gm_pt  Cgs_xm4  Cgd_xm4  Cgd_xm7  Cgs_xm8  Vdd  Rds_xm7  s  Rds_xm1  Cgs_xm2  Cgs_xm1  gm_xm7  v6  Cgd_xm1  R1  Rds_xm3  v3  gm_xm2  v2  v1  Cgs_xm7  Rds_xm4  Cgd_pt  v5  Rds_xm8  Rds_xm2  Cgd_xm2  gm_xm4  Rds_pt ')


In [27]:
print('A =',A)

A = Matrix([[Cgs_pt*s + Cgs_xm3*s + Cgs_xm4*s + 1/Rds_xm4 + 1/Rds_xm3 + 1/Rds_pt, -Cgs_pt*s - gm_pt + gm_xm4 - 1/Rds_xm4, -Cgs_xm3*s - Cgs_xm4*s - gm_xm4 - 1/Rds_xm3, 0, 0, 0, gm_pt - 1/Rds_pt, 1], [-Cgs_pt*s - 1/Rds_xm4, Cgd_pt*s + Cgd_xm2*s + Cgd_xm4*s + Cgs_pt*s - gm_xm2 - gm_xm4 + 1/Rds_xm4 + 1/Rds_xm2, -Cgd_xm4*s + gm_xm4, -1/Rds_xm2, -Cgd_xm2*s + gm_xm2, 0, -Cgd_pt*s, 0], [-Cgs_xm3*s - Cgs_xm4*s - 1/Rds_xm3, -Cgd_xm4*s, Cgd_xm1*s + Cgd_xm4*s + Cgs_xm3*s + Cgs_xm4*s - gm_xm1 + 1/Rds_xm3 + 1/Rds_xm1, -1/Rds_xm1, 0, 0, 0, 0], [0, gm_xm2 - 1/Rds_xm2, gm_xm1 - 1/Rds_xm1, Cgd_xm7*s + Cgs_xm1*s + Cgs_xm2*s - gm_xm7 + 1/Rds_xm7 + 1/Rds_xm2 + 1/Rds_xm1, -Cgs_xm2*s - gm_xm2, -Cgd_xm7*s + gm_xm7, 0, 0], [0, -Cgd_xm2*s, 0, -Cgs_xm2*s, Cgd_xm2*s + Cgs_xm2*s + 1/R2 + 1/R1, 0, -1/R1, 0], [0, 0, 0, -Cgd_xm7*s, 0, Cgd_xm7*s + Cgs_xm7*s + Cgs_xm8*s + 1/Rds_xm8, 0, 0], [-1/Rds_pt, -Cgd_pt*s + gm_pt, 0, 0, -1/R1, 0, Cgd_pt*s - gm_pt + 1/Rds_pt + 1/R1, 0], [1, 0, 0, 0, 0, 0, 0, 0]])


In [28]:
print('X = Matrix(',X,')')

X = Matrix( [v1, v2, v3, v4, v5, v6, v7, I_Vdd] )


In [29]:
print('Z = Matrix(',Z,')')

Z = Matrix( [0, 0, 0, 0, 0, 0, 0, Vdd] )


In [73]:
# print a list of the element values in python dictionary format
values={}
for i in range(len(df)):
    if df.iloc[i]['element'][0] == 'F' or df.iloc[i]['element'][0] == 'E' or df.iloc[i]['element'][0] == 'G' or df.iloc[i]['element'][0] == 'H':
        print('{:s}:{:.4e},'.format(df.iloc[i]['element'].lower(),np.float32(df.iloc[i]['value'])))
    else:
        print('{:s}:{:.4e},'.format(df.iloc[i]['element'],np.float32(df.iloc[i]['value'])))
        values[df.iloc[i]['element']]=np.float32(df.iloc[i]['value'])

Vdd:1.0000e+00,
gm_xm1:1.4166e-05,
Rds_xm1:7.9720e+06,
Cgs_xm1:-1.7089e-15,
Cgd_xm1:-2.8881e-19,
gm_xm2:1.5041e-05,
Rds_xm2:6.8897e+06,
Cgs_xm2:-1.7287e-15,
Cgd_xm2:-9.1657e-19,
gm_xm3:7.0482e-06,
Rds_xm3:5.1806e+06,
Cgs_xm3:-2.3760e-15,
Cgd_xm3:-9.8070e-19,
gm_xm4:7.3886e-06,
Rds_xm4:6.0272e+06,
Cgs_xm4:-2.3750e-15,
Cgd_xm4:-2.8276e-19,
gm_xm7:1.5855e-05,
Rds_xm7:3.2867e+06,
Cgs_xm7:-2.1243e-15,
Cgd_xm7:-4.1679e-17,
gm_xm8:1.6496e-05,
Rds_xm8:5.0562e+06,
Cgs_xm8:-2.1287e-15,
Cgd_xm8:-2.7238e-18,
gm_pt:1.0000e+00,
Rds_pt:1.0000e+00,
Cgs_pt:1.0000e+00,
Cgd_pt:1.0000e+00,
R1:1.0000e+05,
R2:2.0000e+05,


In [31]:
equ

⎡            ⎛                                  1        1        1  ⎞      ⎛ 
⎢ I_Vdd + v₁⋅⎜Cgsₚₜ⋅s + Cgsₓₘ₃⋅s + Cgsₓₘ₄⋅s + ────── + ────── + ─────⎟ + v₂⋅⎜-
⎢            ⎝                                Rdsₓₘ₄   Rdsₓₘ₃   Rdsₚₜ⎠      ⎝ 
⎢                                                                             
⎢                 ⎛             1   ⎞      ⎛                                  
⎢-Cgdₚₜ⋅s⋅v₇ + v₁⋅⎜-Cgsₚₜ⋅s - ──────⎟ + v₂⋅⎜Cgdₚₜ⋅s + Cgdₓₘ₂⋅s + Cgdₓₘ₄⋅s + Cg
⎢                 ⎝           Rdsₓₘ₄⎠      ⎝                                  
⎢                                                                             
⎢                                       ⎛                         1   ⎞      ⎛
⎢                     -Cgdₓₘ₄⋅s⋅v₂ + v₁⋅⎜-Cgsₓₘ₃⋅s - Cgsₓₘ₄⋅s - ──────⎟ + v₃⋅⎜
⎢                                       ⎝                       Rdsₓₘ₃⎠      ⎝
⎢                                                                             
⎢         ⎛          1   ⎞      ⎛          1   ⎞    

In [96]:
gm_xm1 = Symbol('gm_xm1')
gm_xm2 = Symbol('gm_xm2')
gm_xm3 = Symbol('gm_xm3')
gm_xm4 = Symbol('gm_xm4')
gm_xm7 = Symbol('gm_xm7')
gm_xm8 = Symbol('gm_xm8')

Rds_xm1 = Symbol('Rds_xm1')
Rds_xm2 = Symbol('Rds_xm2')
Rds_xm3 = Symbol('Rds_xm3')
Rds_xm4 = Symbol('Rds_xm4')
Rds_xm7 = Symbol('Rds_xm7')
Rds_xm8 = Symbol('Rds_xm8')

Cgs_xm1 = Symbol('Cgs_xm1')
Cgs_xm2 = Symbol('Cgs_xm2')
Cgs_xm3 = Symbol('Cgs_xm3')
Cgs_xm4 = Symbol('Cgs_xm4')
Cgs_xm7 = Symbol('Cgs_xm7')
Cgs_xm8 = Symbol('Cgs_xm8')

Cgd_xm1 = Symbol('Cgd_xm1')
Cgd_xm2 = Symbol('Cgd_xm2')
Cgd_xm3 = Symbol('Cgd_xm3')
Cgd_xm4 = Symbol('Cgd_xm4')
Cgd_xm7 = Symbol('Cgd_xm7')
Cgd_xm8 = Symbol('Cgd_xm8')

In [97]:
Cgs_xm3, v7, Cgs_pt, v4, R2, gm_xm1, I_Vdd, gm_pt, Cgs_xm4, Cgd_xm4, Cgd_xm7, Cgs_xm8, Vdd, Rds_xm7, s, Rds_xm1, Cgs_xm2, Cgs_xm1, gm_xm7, v6, Cgd_xm1, 
R1, Rds_xm3, v3, gm_xm2, v2, v1, Cgs_xm7, Rds_xm4, Cgd_pt, v5, Rds_xm8, Rds_xm2, Cgd_xm2, gm_xm4, Rds_pt =
symbols(' Cgs_xm3  v7  Cgs_pt  v4  R2  gm_xm1  I_Vdd  gm_pt  Cgs_xm4  Cgd_xm4  Cgd_xm7  Cgs_xm8  Vdd  Rds_xm7  s  Rds_xm1  
Cgs_xm2  Cgs_xm1  gm_xm7  v6  Cgd_xm1  R1  Rds_xm3  v3  gm_xm2  v2  v1  Cgs_xm7  Rds_xm4  Cgd_pt  v5  Rds_xm8  Rds_xm2  Cgd_xm2  gm_xm4  Rds_pt ')

In [104]:
U=A.subs(
    {
gm_xm1:1.4166e-05,
Rds_xm1:7.9720e+06,
Cgs_xm1:-1.7089e-15,
Cgd_xm1:-2.8881e-19,
gm_xm2:1.5041e-05,
Rds_xm2:6.8897e+06,
Cgs_xm2:-1.7287e-15,
Cgd_xm2:-9.1657e-19,
gm_xm3:7.0482e-06,
Rds_xm3:5.1806e+06,
Cgs_xm3:-2.3760e-15,
Cgd_xm3:-9.8070e-19,
gm_xm4:7.3886e-06,
Rds_xm4:6.0272e+06,
Cgs_xm4:-2.3750e-15,
Cgd_xm4:-2.8276e-19,
gm_xm7:1.5855e-05,
Rds_xm7:3.2867e+06,
Cgs_xm7:-2.1243e-15,
Cgd_xm7:-4.1679e-17,
gm_xm8:1.6496e-05,
Rds_xm8:5.0562e+06,
Cgs_xm8:-2.1287e-15,
Cgd_xm8:-2.7238e-18,
R1:1.0000e+05,
R2:2.0000e+05,
    }
)

In [105]:
equ = Eq(U*Matrix(X),Matrix(Z))
equ

⎡                          ⎛                                                1 
⎢               I_Vdd + v₁⋅⎜Cgsₚₜ⋅s - 4.751e-15⋅s + 3.58942355452615e-7 + ────
⎢                          ⎝                                              Rdsₚ
⎢                                                                             
⎢-Cgdₚₜ⋅s⋅v₇ + v₁⋅(-Cgsₚₜ⋅s - 1.65914520838864e-7) + v₂⋅(Cgdₚₜ⋅s + Cgsₚₜ⋅s - 1
⎢                                                                             
⎢                                    2.8276e-19⋅s⋅v₂ + v₁⋅(4.751e-15⋅s - 1.930
⎢                                                                             
⎢                      1.48958557992365e-5⋅v₂ + 1.40405609633718e-5⋅v₃ + v₄⋅(-
⎢                                                                             
⎢                                                             9.1657e-19⋅s⋅v₂ 
⎢                                                                             
⎢                                                   

In [107]:
solve(equ, X)

KeyboardInterrupt: 